In [1]:
import os
pm_path = os.path.join(os.getenv('appdata'), 'PigiPlus', 'PropertyModel.json')

In [2]:
# load pm config json into big dictionary

import json
with open(pm_path, 'r', encoding='utf-8') as f:
    pm_entities = json.load(f)

In [3]:
# check it worked

for name, entities in pm_entities.items():
    print(f'{len(entities):>5} {name}')

 3128 Properties
   79 PropertyGroups
  101 PropertyGroupAliases
 9147 PropertyGroupProperties
56321 PropertyGroupPropertyAliases
14650 Equations
    3 IndicatorGroups
   12 Indicators
    2 RatioGroups
    5 Ratios
   36 UnitOfMeasureGroups
  134 UnitsOfMeasure
  462 UnitOfMeasureAliases
34727 MeasurementContexts
   59 Codelists
 2317 CodelistEntries
  371 CodelistEntryAliases


In [4]:
# class definitions for types we want to build
from dataclasses import dataclass
from uuid import UUID
from typing import List, Optional

@dataclass(frozen=True)
class ConversionParameters:
    a: float
    b: float
    c: float
    d: float

@dataclass(frozen=True)
class Uom:
    group: str
    name: str
    symbol: str
    guid: UUID  # keeping guid as symbols are not unique
    is_base: bool
    conv_params: List[ConversionParameters]
        
@dataclass(frozen=True)
class Ratio:
    symbol: str
    conv_params: List[ConversionParameters]

@dataclass(frozen=True)
class PropertyMetadata:
    indicator: str
    default_uom: Uom
    default_ratio: Ratio
    abs_min: Optional[float]
    abs_max: Optional[float]

@dataclass(frozen=True)
class Property:
    analysis_short_name: str
    short_name: str
    internal_name: str
    full_name: str
    guid: UUID
    data_type: str
    # Todo incl alaises? desc?, ion_channel?
    position: float
    metadata: List[PropertyMetadata]

In [5]:
# load uoms

from typing import Dict

guid_to_uom_grp_name: Dict[str, str] = {}
for g in pm_entities["UnitOfMeasureGroups"]:
    guid_to_uom_grp_name[g["UnitOfMeasureGroupId"]] = g["Name"]

guid_to_uom: Dict[str, Uom] = {}
for u in pm_entities["UnitsOfMeasure"]:
    p = u["ConversionParameters"]
    params = ConversionParameters(p["A"], p["B"], p["C"], p["D"])
    uom = Uom(
        group = guid_to_uom_grp_name[u["UnitOfMeasureGroupId"]],
        name = u["LongName"],
        symbol = u["Symbol"],
        guid = UUID(u["UnitOfMeasureId"]),
        is_base = u["IsBase"],
        conv_params = params)
    guid_to_uom[u["UnitOfMeasureId"]] = uom

In [6]:
# load ratios

guid_to_ratio: Dict[str, str] = {}
for r in pm_entities["Ratios"]:
    p = r["ConversionParameters"]
    params = ConversionParameters(p["A"], p["B"], p["C"], p["D"])
    ratio = Ratio(r["Symbol"], params)
    guid_to_ratio[r["RatioId"]] = ratio  

In [7]:
# todo - can't find mappings from GUID yet
#guid_to_data_type: Dict[str, str] = {   # Hard coding as they don't seem to be in the json
  
#}

In [8]:
# load properties

from collections import defaultdict, namedtuple
    
guid_to_ind: Dict[str, str] = {}
for i in pm_entities["Indicators"]:
    guid_to_ind[i["IndicatorId"]] = i["ShortName"]

pgp_guid_to_metadatas: Dict[str, PropertyMetadata] = defaultdict(list)
for mc in pm_entities["MeasurementContexts"]:
    metadata = PropertyMetadata(
        indicator = guid_to_ind[mc["IndicatorId"]],
        default_uom = guid_to_uom[mc["DefaultValueDescriptorUnitOfMeasureId"]],
        default_ratio = guid_to_ratio[mc["DefaultValueDescriptorRatioId"]],
        abs_min = mc["AbsoluteMin"],
        abs_max = mc["AbsoluteMax"])
    pgp_guid = mc["PropertyGroupPropertyId"]
    pgp_guid_to_metadatas[pgp_guid].append(metadata)

pg_short_name_by_guid: Dict[str, str] = {}
for pg in pm_entities["PropertyGroups"]:
    pg_short_name_by_guid[pg["PropertyGroupId"]] = pg["ShortName"]
    
PropConfig = namedtuple('PropConfig', 'name long_name internal_name data_type_guid')
guid_to_prop_config: Dict[str, PropConfig] = {}
for p in pm_entities["Properties"]:
    config = PropConfig(p["ShortName"], p["LongName"], p["IgiCode"], p["DataTypeId"])
    guid_to_prop_config[p["PropertyId"]] = config

properties: List[Property] = []
for p in pm_entities["PropertyGroupProperties"]:
    prop_config = guid_to_prop_config[p["PropertyId"]]
    prop = Property(
        analysis_short_name = pg_short_name_by_guid[p["PropertyGroupId"]],
        short_name = prop_config.name,
        internal_name = prop_config.internal_name,
        full_name = prop_config.long_name,
        guid = UUID(p["PropertyGroupPropertyId"]),
        data_type = prop_config.data_type_guid,  # TODO: replace with mapping from guid
        position = p["PositionInPropertyGroup"],
        metadata = pgp_guid_to_metadatas[p["PropertyGroupPropertyId"]])
    properties.append(prop)

In [9]:
len(properties)

9147

In [10]:
len(pm_entities["PropertyGroupProperties"])

9147

In [11]:
properties[0]

Property(analysis_short_name='WO-GC', short_name='nC20', internal_name='nC20', full_name='n-Eicosane', guid=UUID('07cd6737-01f5-4d74-9d0a-00b2115efa2d'), data_type='c0305e1d-c8a1-4767-8b7f-f5363bb1dbe2', position=134, metadata=[PropertyMetadata(indicator='h', default_uom=Uom(group='Unitless GC', name='Count', symbol='count', guid=UUID('58f35f5e-dfd8-4797-aab9-6698bf612bf5'), is_base=True, conv_params=ConversionParameters(a=1.0, b=0.0, c=0.0, d=1.0)), default_ratio=Ratio(symbol='NoRatio', conv_params=ConversionParameters(a=1.0, b=0.0, c=0.0, d=1.0)), abs_min=None, abs_max=None), PropertyMetadata(indicator='a', default_uom=Uom(group='Unitless GC', name='Count', symbol='count', guid=UUID('58f35f5e-dfd8-4797-aab9-6698bf612bf5'), is_base=True, conv_params=ConversionParameters(a=1.0, b=0.0, c=0.0, d=1.0)), default_ratio=Ratio(symbol='NoRatio', conv_params=ConversionParameters(a=1.0, b=0.0, c=0.0, d=1.0)), abs_min=None, abs_max=None), PropertyMetadata(indicator='ch', default_uom=Uom(group='Ma